In [26]:
BASE_DIR = '.'
DATA_DIR = f'{BASE_DIR}/miniResults'

from util import count_files

train_text_path = f"{DATA_DIR}/train_text"
train_time_path = f"{DATA_DIR}/train_timestamps"
train_price_path = f"{DATA_DIR}/train_price"
train_mask_path = f"{DATA_DIR}/train_mask"
train_gt_path = f"{DATA_DIR}/train_gt"
no_of_train_samples = count_files(train_text_path)

val_text_path = f"{DATA_DIR}/val_text"
val_time_path = f"{DATA_DIR}/val_timestamps"
val_price_path = f"{DATA_DIR}/val_price"
val_mask_path = f"{DATA_DIR}/val_mask"
val_gt_path = f"{DATA_DIR}/val_gt"
no_of_validation_samples = count_files(val_text_path)

test_text_path = f"{DATA_DIR}/test_text"
test_time_path = f"{DATA_DIR}/test_timestamps"
test_price_path = f"{DATA_DIR}/test_price"
test_mask_path = f"{DATA_DIR}/test_mask"
test_gt_path = f"{DATA_DIR}/test_gt"

no_of_test_samples = count_files(test_text_path)

no_stocks = 2

In [2]:
import numpy as np
import torch

i = 0
device = 'mps'
train_text = torch.tensor(np.load(train_text_path+str(i).zfill(10)+'.npy'), dtype=torch.float32).to(device)
train_timestamps = torch.tensor(np.load(train_time_path+str(i).zfill(10)+'.npy'), dtype=torch.float32).to(device)
mask_batch = (np.load(train_mask_path+str(i).zfill(10)+'.npy'))#.to(device)
price_batch = (np.load(train_price_path+str(i).zfill(10)+'.npy'))#.to(device)
gt_batch = (np.load(train_gt_path+str(i).zfill(10)+'.npy'))#.to(device)

In [9]:
price_batch, mask_batch, gt_batch

(array([[0.82422],
        [0.67907]]),
 array([[1.],
        [0.]]),
 array([[0.131],
        [0.006]]))

In [31]:
from torch.utils.data import Dataset, DataLoader

class StockData(Dataset):
    def __init__(self, paths, loopback_window=3, mask_threshold=0.5, device='cpu'):
        self.mask_threshold = mask_threshold
        self.window = loopback_window
        self.paths = paths
        self.device = device
        

    # def __len__(self):
    #     return len(self.data)

    def __getitem__(self, idx):
        train_text_path = self.paths['text']
        train_time_path = self.paths['time']
        train_price_path = self.paths['price']
        train_gt_path = self.paths['gt']

        train_text, train_timestamps = self._get_days_lookback(idx, self.window)
        
        price_batch = self._load_np(train_price_path, idx)
        gt_batch = self._load_np(train_gt_path, idx)
        
        mask_batch = self._get_mask_batch(price_batch)

        return (
            train_text, train_timestamps, price_batch, gt_batch, mask_batch
        )    


    def _load_np(self, pathdir, i):
        return np.load(f"{pathdir}/{str(i).zfill(10)}.npy")
    
    def _load_torchf(self, pathdir, i):
        a = self._load_np(pathdir, i)
        return torch.tensor(a, dtype=torch.float32).to(self.device)
    
    def _get_mask_batch(self, price_batch):
        price_batch = price_batch.astype(float)
        return (price_batch > self.mask_threshold).astype(int)
    
    def _get_day_texts(self, day_idx):
        return self._load_torchf(self.paths['text'], day_idx)
    
    def _get_day_timestamps(self, day_idx):
        return self._load_torchf(self.paths['time'], day_idx)
    
    def _get_days_lookback(self, day_idx, window):
        # get tweets and timestamps in the recent days
        texts =  torch.vstack([
            self._get_day_texts(day) for day in range(max(0, day_idx-window), day_idx+1)
        ])
        times =  torch.vstack([
            self._get_day_timestamps(day) for day in range(max(0, day_idx-window), day_idx+1)
        ])
        return texts, times
    
    def to(self, device):
        self.device = device


SyntaxError: invalid syntax (408050679.py, line 62)

In [ ]:
class AllStocksDataset(Dataset):
    def __init__(self, paths, loopback_window=3, mask_threshold=0.5, device='cpu'):
        self.mask_threshold = mask_threshold
        self.window = loopback_window
        self.paths = paths
        self.device = device



    # def __len__(self):
    #     return len(self.data)

    def __getitem__(self, stock_idx):

In [28]:

paths = {
    'text': f'{BASE_DIR}/Train',
    'time': train_time_path,
    'price': train_price_path,
    'gt': train_gt_path,
}

dataset = CustomDataset(paths, 3, 0.5, 'mps')

In [29]:
texts, times, _, _, _ = dataset[2]

FileNotFoundError: [Errno 2] No such file or directory: './miniResults/train_text/0000000000.npy'

In [30]:
texts.shape, times.shape

(torch.Size([6, 5, 10, 768]), torch.Size([6, 5, 10, 1]))

In [51]:
import os

def listdir_sorted(path):
    if isinstance(path, list):
        path = os.path.join(*path)
    
    dirs = os.listdir(path)
    return sorted(dirs)


train_dir = "Training"

stocks = os.listdir(train_dir)
# stocks = ['AAPL', 'GOOG', 'MSFT']
stocks = ['AAPL']

for stock in stocks:
    texts = listdir_sorted([train_dir, stock, 'text'])
    times = listdir_sorted([train_dir, stock, 'time'])

In [52]:
texts, times

(['2014-01-01_embeddings.npy',
  '2014-01-02_embeddings.npy',
  '2014-01-03_embeddings.npy',
  '2014-01-04_embeddings.npy',
  '2014-01-05_embeddings.npy',
  '2014-01-06_embeddings.npy',
  '2014-01-07_embeddings.npy',
  '2014-01-08_embeddings.npy',
  '2014-01-09_embeddings.npy',
  '2014-01-10_embeddings.npy',
  '2014-01-11_embeddings.npy',
  '2014-01-12_embeddings.npy',
  '2014-01-13_embeddings.npy',
  '2014-01-14_embeddings.npy',
  '2014-01-15_embeddings.npy'],
 ['2014-01-01_its.npy',
  '2014-01-02_its.npy',
  '2014-01-03_its.npy',
  '2014-01-04_its.npy',
  '2014-01-05_its.npy',
  '2014-01-06_its.npy',
  '2014-01-07_its.npy',
  '2014-01-08_its.npy',
  '2014-01-09_its.npy',
  '2014-01-10_its.npy',
  '2014-01-11_its.npy',
  '2014-01-12_its.npy',
  '2014-01-13_its.npy',
  '2014-01-14_its.npy',
  '2014-01-15_its.npy'])

In [102]:
from torch.utils.data import Dataset, DataLoader

# folder hierachy:
# base_dir
#   data_split
#       data_type
 

class StockData(Dataset):
    def __init__(self, base_dir, data_split, stock, loopback_window=3, mask_threshold=0.5, device='cpu'):
        base_dir = os.path.join(base_dir, data_split, stock)
        self.paths = self._get_paths(base_dir)
       
        self.mask_threshold = mask_threshold
        self.window = loopback_window
        self.device = device
        self.stock = stock


    @staticmethod
    def _get_paths(base_dir):
        data_types = ['text', 'time']
        paths = {
            data_type: listdir_sorted([base_dir, data_type]) for data_type in data_types
        }
        return {
            data_type: [os.path.join(base_dir, data_type, path) for path in value]
            for data_type, value in paths.items()
        }

    def __len__(self):
        return len(self.paths['text'])

    def __getitem__(self, idx):
        train_text, train_timestamps = self._get_days_lookback(idx, self.window)
        
        # price_batch = self._get_price(idx)
        # gt_batch = self._get_gt(idx)        
        # mask_batch = self._get_mask_batch(price_batch)

        price_batch = gt_batch = mask_batch = None
        
        return (
            train_text, train_timestamps, price_batch, gt_batch, mask_batch
        )


    def _load_np(self, data_type, i):
        path = self.paths[data_type][i]
        return np.load(path)
    
    def _load_torchf(self, data_type, i):
        a = self._load_np(data_type, i)
        return torch.tensor(a, dtype=torch.float32).to(self.device)
    
    def _get_mask_batch(self, price_batch):
        price_batch = price_batch.astype(float)
        return (price_batch > self.mask_threshold).astype(int)
    
    def _get_day_texts(self, day_idx):
        return self._load_torchf('text', day_idx)
    
    def _get_day_timestamps(self, day_idx):
        return self._load_torchf('time', day_idx)
    
    def _get_days_lookback(self, day_idx, window):
        # get tweets and timestamps in the recent days
        texts =  torch.vstack([
            self._get_day_texts(day) for day in range(max(0, day_idx-window), day_idx+1)
        ])
        times =  torch.vstack([
            self._get_day_timestamps(day) for day in range(max(0, day_idx-window), day_idx+1)
        ])
        return texts, times
    
    def _get_price(self, day_idx):
        return self._load_np('price', day_idx)
    def _get_gt(self, day_idx):
        return self._load_np('gt', day_idx)

    def to(self, device):
        self.device = device

In [103]:
ds = StockData('data', 'Training', 'AAPL')

In [106]:
ds[0]

(tensor([[ 0.2114,  0.0748,  0.4601,  ..., -0.2690,  0.0447,  0.1695],
         [ 0.2114,  0.0748,  0.4601,  ..., -0.2690,  0.0447,  0.1695],
         [ 0.0373,  0.2427,  0.6379,  ..., -0.2408,  0.1220,  0.1625],
         ...,
         [-0.0997,  0.2427,  0.7195,  ..., -0.3838, -0.0209,  0.2701],
         [ 0.1067,  0.1702,  0.3629,  ..., -0.4273,  0.1017,  0.3147],
         [-0.0028,  0.1038,  0.5053,  ..., -0.1084, -0.0363,  0.0812]]),
 tensor([[1.0000, 1.0000, 0.0078, 0.0295, 0.0103, 0.0338, 0.0024, 0.0040, 0.0053]]),
 None,
 None,
 None)

In [105]:
ds.paths

{'text': ['data/Training/AAPL/text/2014-01-01_embeddings.npy',
  'data/Training/AAPL/text/2014-01-02_embeddings.npy',
  'data/Training/AAPL/text/2014-01-03_embeddings.npy',
  'data/Training/AAPL/text/2014-01-04_embeddings.npy',
  'data/Training/AAPL/text/2014-01-05_embeddings.npy',
  'data/Training/AAPL/text/2014-01-06_embeddings.npy',
  'data/Training/AAPL/text/2014-01-07_embeddings.npy',
  'data/Training/AAPL/text/2014-01-08_embeddings.npy',
  'data/Training/AAPL/text/2014-01-09_embeddings.npy',
  'data/Training/AAPL/text/2014-01-10_embeddings.npy',
  'data/Training/AAPL/text/2014-01-11_embeddings.npy',
  'data/Training/AAPL/text/2014-01-12_embeddings.npy',
  'data/Training/AAPL/text/2014-01-13_embeddings.npy',
  'data/Training/AAPL/text/2014-01-14_embeddings.npy',
  'data/Training/AAPL/text/2014-01-15_embeddings.npy'],
 'time': ['data/Training/AAPL/time/2014-01-01_its.npy',
  'data/Training/AAPL/time/2014-01-02_its.npy',
  'data/Training/AAPL/time/2014-01-03_its.npy',
  'data/Traini